In [2]:
import pandas as pd 
import numpy as np

In [12]:
#diese Daten müssen an den Datensatz angepast werden
train="data/affen2train.csv"
test="data/affen2test.csv"
spalten=["Nummer","Auge_li","Auge_re","Mund","Beschriftung"]
#train="data/PlayTennisTrain.csv"
#test="data/PlayTennisTest.csv"
#spalten=[0,1,2,3,4,5]
sep=';'
label="Beschriftung"
train_data_m = pd.read_csv(train,sep=sep,usecols=spalten)
train_data_m.head()

,Nummer,Auge_li,Auge_re,Mund,Beschriftung
0,3,x,x,Zahn,B
1,5,offen,offen,Grinsen,B
2,8,zu,zu,Grinsen,B
3,11,x,x,grimmig,B
4,15,x,x,weit offen,B


In [13]:
train_data=train_data_m.copy()
class_list = train_data[label].unique()
class_list

array(['B', 'BN'], dtype=object)

Für alle Attribute (Auge_li, Auge_re, Mund) muss der Informationsgewinn Gain(A,S) berechnet werden. Das ist der Wert, der die Reduzierung der Unsicherheit (Entropie) beschreibt, wenn man den Wert von A kennt. 

$$ Gain(S,A) = H(S) - \sum_{v \in  A} \frac{|S_v|}{|S|} H(S_v)$$
S   : Datensatz  
H(S): Entropie des Datensatz  
$S_v$ : Teilmenge von S für den A den Wert v hat   
|S| : Mächtigkeit von S  
$|S_v|$: Mächtigkeit von $S_v$
So berechnet man die Entropie H(S)  
$$ H(S) = -\sum_{c=1}^{|C|} p_clog_2 p_c$$
|C|  : Anzahl der Kategorien  
$p_c$: Anteil der Instanzen in S, die der Kategorie c angehören

In [14]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]
        #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy
total_entropy=calc_entropy(train_data, label, class_list)#Gesamtentropie
print("Gesamtentropie:", total_entropy)

def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        #Zeilen auswählen mit feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list)
        #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy
        #aufaddieren
    info_gain=    calc_entropy(train_data, label, class_list) - feature_info
    print(feature_name,info_gain)
    return info_gain
        #calculating information gain by subtracting
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label)
    #finding the feature names in the dataset
    #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain:
            #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature  
    print(max_info_feature)
    return max_info_feature
#find_most_informative_feature(train_data,label,class_list)


Gesamtentropie: 0.9852281360342515


In [15]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False)
    #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]
            #count of class c

            if class_count == count: #count of feature_value = count of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value]
                #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #should extend the node, so the branch is marked with ?
            
    return tree, train_data
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list)
        #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list)
        #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node]
                #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list)
                #recursive call with updated dataset

def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data_m, label, class_list) #start calling recursion
    return tree
tree=id3(train_data,label)
print("der Baum",tree)

def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_preditct += 1 #increase correct count
        else:
            wrong_preditct += 1 #increase incorrect count
    print("richtig: ",correct_preditct,"  falsch: ",wrong_preditct )
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
    return accuracy

test_data_m = pd.read_csv(test,sep=sep,usecols=spalten)
#importing test dataset into dataframe
accuracy = evaluate(tree, test_data_m, label) #evaluating the test dataset
print(accuracy)

Nummer 0.9852281360342515
Auge_li 0.49261406801712576
Auge_re 0.49261406801712576
Mund 0.19811742113040343
Nummer
der Baum {'Nummer': {3: 'B', 5: 'B', 8: 'B', 11: 'B', 15: 'B', 19: 'B', 1: 'BN', 2: 'BN', 10: 'BN', 12: 'BN', 13: 'BN', 14: 'BN', 18: 'BN', 20: 'BN'}}
richtig:  0   falsch:  6
0.0
